In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collection import collections
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from webUtils import getHTML
from time import sleep
import time
from multiprocessing import Pool


################################################################################
## Music Stuff
################################################################################

## General
from discogsBase import discogs

## Discogs
from artist import artist
from artists import artists
from discogsUtils import discogsUtils
from albums import albums

## AllMusic
from artistAM import artistAM
from artistsAM import artistsAM
from discogsUtils import allmusicUtils
from artistsAM import artistsAM

## MusicBrainz
from artistMB import artistMB
from artistsMB import artistsMB
from discogsUtils import musicbrainzUtils

## AceBootlegs
from artistAB import artistAB
from artistsAB import artistsAB
from discogsUtils import acebootlegsUtils

## RateYourMusic
from artistRM import artistRM
from artistsRM import artistsRM
from discogsUtils import rateyourmusicUtils

## LastFM
from artistLM import artistLM
from artistsLM import artistsLM
from discogsUtils import lastfmUtils



import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-03-25 18:21:57.327993


In [2]:
%load_ext autoreload
%autoreload
discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artists(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

discAB   = discogs('acebootlegs')
artsAB   = artistsAB(discAB)
dutilsAB = acebootlegsUtils()

discRM   = discogs('rateyourmusic')
artsRM   = artistsRM(discRM)
dutilsRM = rateyourmusicUtils()

discLM   = discogs('lastfm')
artsLM   = artistsLM(discLM)
dutilsLM = lastfmUtils()

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Wed Mar 25, 2020 18:21:59 for Last Run


# Collections

In [3]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parse Artists (All Music)

In [ ]:
for i in range(100):
    artsAM.parseArtistModValFiles(i, force=True)

In [ ]:
artMB = artistMB(discMB)
retval = artMB.getData("/Volumes/Biggy/Discog/artists-musicbrainz/0/148150553282083715092252970758494088400.p")
retval.url.get()
retval.ID.get()

# Parse Artists (Parallel)

In [18]:
%load_ext autoreload
%autoreload
discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artists(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

discAB   = discogs('acebootlegs')
artsAB   = artistsAB(discAB)
dutilsAB = acebootlegsUtils()

discRM   = discogs('rateyourmusic')
artsRM   = artistsRM(discRM)
dutilsRM = rateyourmusicUtils()

discLM   = discogs('lastfm')
artsLM   = artistsLM(discLM)
dutilsLM = lastfmUtils()

def parseArtistsAM(modVal, force=False, doExtra=False):
    artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=False, doExtra=False):
    artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    artsMB.parseArtistModValFiles(modVal, force=force)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    artsLM.parseArtistModValFiles(modVal, force=force)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "RateYourMusic":
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#artsDC.parseArtistModValFiles(29, force=True)
#arts.parseArtistModValFiles(50, force=True)
#artsLM.parseArtistModValFiles(96)

In [21]:
ifile="/Volumes/Biggy/Discog/artists-lastfm/0/53902704000.p"
bsdata = getHTML(ifile)
artLM = artistLM()
#artLM.getData(ifile)

In [ ]:
#bsdata

In [17]:
#for modVal in range(100):
#    parseArtistsLM(modVal, force=True)

In [19]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
#parseArtistsParallel(db="MusicBrainz", nProcs=3)
#parseArtistsParallel(db="Discogs", nProcs=3)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=1)
#parseArtistsParallel(db="LastFM", nProcs=3)
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Wed Mar 25, 2020 18:29:07 for Parsing All Artists
Current Time is Wed Mar 25, 2020 18:29:07 for Parallel Artist Parsing
Parsing Artist Files For ModVal 0
Forcing Reloads of ModVal=0
  Processing 0 files.
Parsing Artist Files For ModVal 1
Forcing Reloads of ModVal=1
  Processing 0 files.
Parsing Artist Files For ModVal 2
Forcing Reloads of ModVal=2
  Processing 0 files.
Parsing Artist Files For ModVal 3
Forcing Reloads of ModVal=3
  Processing 0 files.
Parsing Artist Files For ModVal 4
Forcing Reloads of ModVal=4
  Processing 0 files.
Parsing Artist Files For ModVal 5
Forcing Reloads of ModVal=5
  Processing 0 files.
Parsing Artist Files For ModVal 6
Forcing Reloads of ModVal=6
  Processing 0 files.
Parsing Artist Files For ModVal 7
Forcing Reloads of ModVal=7
  Processing 0 files.
Parsing Artist Files For ModVal 8
Forcing Reloads of ModVal=8
  Processing 0 files.
Parsing Artist Files For ModVal 9
Forcing Reloads of ModVal=9
  Processing 0 files.
Parsing Artist Files For

Forcing Reloads of ModVal=91
  Processing 0 files.
Parsing Artist Files For ModVal 92
Forcing Reloads of ModVal=92
  Processing 0 files.
Parsing Artist Files For ModVal 93
Forcing Reloads of ModVal=93
  Processing 0 files.
Parsing Artist Files For ModVal 94
Forcing Reloads of ModVal=94
  Processing 0 files.
Parsing Artist Files For ModVal 95
Forcing Reloads of ModVal=95
  Processing 0 files.
Parsing Artist Files For ModVal 96
Forcing Reloads of ModVal=96
  Processing 0 files.
Parsing Artist Files For ModVal 97
Forcing Reloads of ModVal=97
  Processing 0 files.
Parsing Artist Files For ModVal 98
Forcing Reloads of ModVal=98
  Processing 0 files.
Parsing Artist Files For ModVal 99
Forcing Reloads of ModVal=99
  Processing 0 files.

Current Time is Wed Mar 25, 2020 18:29:08 for Done with Parallel Artist Parsing
Process [Done with Parallel Artist Parsing] took 1 seconds.
Current Time is Wed Mar 25, 2020 18:29:08 for Done with Parsing All Artists
Process [Done with Parsing All Artists] took

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
#albs.parseAlbumModValFiles(modVal=27)

In [ ]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 2
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

In [ ]:
from fsUtils import removeFile